# Challenge 0

In [2]:
!pip freeze

absl-py==0.5.0
adal==1.1.0
alabaster==0.7.10
alembic==1.0.0
anaconda-client==1.6.14
anaconda-project==0.8.2
appdirs==1.4.3
applicationinsights==0.11.6
asn1crypto==0.24.0
astor==0.7.1
astroid==1.6.3
astropy==3.0.2
async-generator==1.10
attrs==18.1.0
audioread==2.1.6
autovizwidget==0.12.5
azure==4.0.0
azure-applicationinsights==0.1.0
azure-batch==4.1.3
azure-cognitiveservices-nspkg==2.0.0
azure-cognitiveservices-vision-customvision==0.3.0
azure-cognitiveservices-vision-nspkg==2.0.0
azure-common==1.1.15
azure-cosmosdb-nspkg==2.0.2
azure-cosmosdb-table==1.0.5
azure-datalake-store==0.0.31
azure-eventgrid==1.2.0
azure-graphrbac==0.40.0
azure-keyvault==1.1.0
azure-loganalytics==0.1.0
azure-mgmt==4.0.0
azure-mgmt-advisor==1.0.1
azure-mgmt-applicationinsights==0.1.1
azure-mgmt-authorization==0.50.0
azure-mgmt-batch==5.0.1
azure-mgmt-batchai==2.0.0
azure-mgmt-billing==0.2.0
azure-mgmt-cdn==3.0.0
azure-mgmt-cognitiveservices==3.0.0
azure-mgmt-commerce==1.0.1
azure-mgmt-compute==4.1.0
azure-mgmt-c

In [3]:
!pip --version

pip 18.0 from /anaconda/envs/py35/lib/python3.5/site-packages/pip (python 3.5)


In [4]:
#! curl -O https://challenge.blob.core.windows.net/challengefiles/gear_images.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 33.4M  100 33.4M    0     0  6463k      0  0:00:05  0:00:05 --:--:-- 7859k


# Challenge 1

# Challenge 2

In [91]:
import numpy as np
from PIL import Image
def load_image(path_to_img):
    '''
        Load images and resize them if they are squared and not in 128x128x3
        Input:
            - path_to_img (string): local path to an image
        Output:
            - img (PIL.Image): Image in corrected size.
    '''
    img = Image.open(path_to_img)
    
    if(img.size[0] > img.size[1]):
        white = np.zeros((img.size[0] - img.size[1], img.size[0],  3)) + 255
        img = np.vstack((np.array(img)[:,:,:3], white))
    elif(img.size[0] < img.size[1]):
        white = np.zeros((img.size[1], img.size[1] - img.size[0], 3)) + 255
        img = np.hstack((np.array(img)[:,:,:3], white))
    else:
        img = np.array(img)[:,:,:3]
    img = Image.fromarray(np.uint8(img))
    basewidth = 128
    wpercent = (basewidth/float(img.size[0]))
    hsize = int((float(img.size[1])*float(wpercent)))
    img = img.resize((basewidth,hsize), Image.ANTIALIAS)
    return img

In [92]:
import cv2

In [93]:
import os

for element_type in os.listdir('./gear_images/'):
    for pic in os.listdir('./gear_images/' + element_type):
        try:
            img = load_image('./gear_images/' + element_type +'/'+ pic)
            hist,bins = np.histogram(np.array(img).flatten(),256,[0,256])
            cdf = hist.cumsum()
            cdf_normalized = cdf * hist.max()/ cdf.max()
            cdf_m = np.ma.masked_equal(cdf,0)
            cdf_m = (cdf_m - cdf_m.min())*255/(cdf_m.max()-cdf_m.min())
            cdf = np.ma.filled(cdf_m,0).astype('uint8')
            img2 = cdf[img]

            img = Image.fromarray(img2)
            img.save('./resized_img/' + element_type + '/' + pic)
        except IsADirectoryError:
            print('Notebook path')
            

Notebook path


# Challenge 3

## Using sklearn to predict classes``

In [6]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
import pandas as pd
import os
from PIL import Image
import numpy as np
from sklearn.preprocessing import StandardScaler

In [2]:
labels = []
list_imgs = []
for element_type in os.listdir('./resized_img//'):
    for pic in os.listdir('./resized_img//' + element_type):
        try:
            img = Image.open('./resized_img/' + element_type + '/' + pic)
            img = np.array(img).flatten()
            labels.append(element_type)
            list_imgs.append(img)
        except IsADirectoryError:
            print(".ipynb_checkpoints error")

In [3]:
len(labels)

2122

In [22]:
images = np.array(list_imgs)

In [8]:
scaler = StandardScaler()
scaler.fit(images)

images = scaler.transform(images)

/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [23]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [24]:
from sklearn.ensemble import ExtraTreesClassifier

In [25]:
%%time
model = ExtraTreesClassifier()
model.fit(X_train, y_train)

CPU times: user 901 ms, sys: 67.3 ms, total: 968 ms
Wall time: 967 ms


In [26]:
model.score(X_test, y_test)

0.8470588235294118

# Challenge 4

Suggested architecture:
<br/>
Input Layer (3 channel image input layer)<br/>
Convolutional (2D)<br/>
Max Poolingv<br/>
Convolutional (2D)<br/>
Max Pooling<br/>
Dense (Output layer)<br/>

In [108]:
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.core import Dense
from keras.layers.core import Activation
from keras.layers.convolutional import MaxPooling2D
from keras.utils import to_categorical
from keras import optimizers
from keras.layers.core import Flatten
from keras.layers import Dropout
from keras.callbacks import EarlyStopping

In [47]:
labels[0]

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0])

In [77]:
labels = []
list_imgs = []
for element_type in os.listdir('./resized_img//'):
    for pic in os.listdir('./resized_img//' + element_type):
        try:
            img = Image.open('./resized_img/' + element_type + '/' + pic)
            img = np.array(img)
            labels.append(element_type)
            list_imgs.append(img)
        except IsADirectoryError:
            print(".ipynb_checkpoints error")

In [78]:
images = np.array(list_imgs)

In [79]:
images.shape

(2122, 128, 128, 3)

In [80]:
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
lb.fit(labels)

LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)

In [81]:
labels = lb.transform(labels)

In [103]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [104]:
X_train = X_train/255
X_test = X_test/255

In [158]:
model = Sequential()
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)))
model.add(Conv2D(32, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(12, activation = 'softmax'))

In [159]:
early = EarlyStopping(monitor='val_loss', min_delta=0, patience=4, verbose=0, mode='auto', baseline=None)
model.compile(optimizer=optimizers.SGD(), loss='categorical_crossentropy', metrics=['acc'])

In [160]:
model.fit(X_train, y_train, batch_size=32, epochs=20, validation_split=0.1, callbacks=[early])

Train on 1527 samples, validate on 170 samples
Epoch 1/20
1527/1527 [==============================] - 153s 100ms/step - loss: 1.9977 - acc: 0.3268 - val_loss: 1.8494 - val_acc: 0.3118
Epoch 2/20
1527/1527 [==============================] - 237s 156ms/step - loss: 1.2393 - acc: 0.5776 - val_loss: 0.9704 - val_acc: 0.6647
Epoch 3/20
1527/1527 [==============================] - 235s 154ms/step - loss: 0.9566 - acc: 0.6791 - val_loss: 0.7207 - val_acc: 0.7588
Epoch 4/20
1527/1527 [==============================] - 237s 155ms/step - loss: 0.7931 - acc: 0.7243 - val_loss: 0.7942 - val_acc: 0.7176
Epoch 5/20
1527/1527 [==============================] - 241s 158ms/step - loss: 0.6664 - acc: 0.7629 - val_loss: 0.6301 - val_acc: 0.8235
Epoch 6/20
1527/1527 [==============================] - 239s 157ms/step - loss: 0.5933 - acc: 0.7957 - val_loss: 0.5430 - val_acc: 0.8294
Epoch 7/20
1024/1527 [===================>..........] - ETA: 1:16 - loss: 0.5377 - acc: 0.8047

KeyboardInterrupt: 

In [157]:
model.evaluate(x=X_test, y=y_test)

425/425 [==============================] - 3s 8ms/step


[0.46173039075206307, 0.8494117647058823]

In [117]:
y_pred = model.predict(X_test)

In [123]:
y_pred2 = np.zeros(y_pred.shape)

In [131]:
count = 0
for a in y_pred:
    
    m = np.argmax(a)
    y_pred2[count][m] = 1
    count+=1

In [141]:
score = 0
for i in range(0, 425):
    try:
        np.testing.assert_array_almost_equal(y_test[i], y_pred2[i])
        score+=1
    except AssertionError:
        pass

In [146]:
y_test.shape

(425, 12)

In [147]:
y_pred2.shape

(425, 12)

In [142]:
score = score / 425

In [143]:
score

0.10588235294117647